In [5]:
pgn_path = "./LichessDB/lichess_db_standard_rated_2016-03.pgn"
output_path = "./filtered_positions.txt"
def count_games(pgn_path):
    count = 0
    with open(pgn_path, "r", encoding="utf-8") as f:
        for line in f:
            if line.startswith("[Termination"):
                count += 1
    return count

num_games = count_games(pgn_path)
print(f"Liczba partii: {num_games}")

Liczba partii: 5801234


In [ ]:
import chess
import chess.pgn
import chess.engine
from tqdm import tqdm


engine = chess.engine.SimpleEngine.popen_uci("/usr/games/stockfish")


gameSets = set()
with open(pgn_path) as pgn, open(output_path, "w") as outfile:
    for _ in tqdm(range(num_games), desc="Processing games"):
        game = chess.pgn.read_game(pgn)
        if game is None:
            break
        
        board = game.board()
        moves = []
        
        for i, move in enumerate(game.mainline_moves()):
            board.push(move)
            moves.append(move.uci())
            if i == 11:
                info = engine.analyse(board, chess.engine.Limit(time=0.2))
                score = info["score"].white() 
                
                if not score.is_mate():
                    cp = score.score()
                    if cp is not None and abs(cp) <= 20:
                        moves_str = " ".join(moves)
                        fen_string = board.fen()
                        if(fen_string in gameSets):
                            continue
                        else:
                            outfile.write(f"{fen_string}\n")
                            gameSets.add(fen_string)
                break


engine.quit()

Processing games:   0%|          | 706/5801234 [02:21<321:53:48,  5.01it/s]


KeyboardInterrupt: 